In [11]:
import sys
# setting path
sys.path.append('../')

In [12]:
import cv2
from  pytorchyolo import detect, models_split_tiny
from pytorchyolo.utils.transforms import Resize, DEFAULT_TRANSFORMS
import torchvision.transforms as transforms
import numpy as np
from pytorchyolo.utils.utils import load_classes, rescale_boxes, non_max_suppression, print_environment_info
import pandas as pd
import time
import torch
# import torchvision.ops.boxes as bops
# import matplotlib.pyplot as plt
# import datasketches as dsk
# from libs.MinMaxSketchV2 import MinMaxSketchV2 
# from dahuffman import HuffmanCodec
from torchvision import io as tvio
from torchmetrics.detection import MeanAveragePrecision
from torch import tensor
import os
from torch.profiler import profile, record_function, ProfilerActivity

### Load ground truth

In [13]:
def load_ground_truth(video_name):
    frame_labels = []
    df_gt = pd.read_csv("../bdd100k_label/"+video_name+".csv")
    gt_group = df_gt.groupby("frame_id")
    for key in gt_group.groups.keys():
        df = gt_group.get_group(key)
        labels = []
        boxes = []
        for i in range(len(df)):
            row = df.iloc[i].to_list()
            boxes.append(row[4:])
            labels.append(row[2])

        frame_labels.append(dict(boxes=tensor(boxes,dtype=torch.float32),labels=tensor(labels,dtype=torch.int32),) )
    return frame_labels

### Prepare test frames

In [14]:
def load_video_frames(video_dir,video_name, samples_number=-1): #samples_number = -1 to load all frames
    video_path = video_dir + video_name + ".mov"

    test_frames = []

    cap = cv2.VideoCapture(video_path) # origin frame rate is 30 fps
    exist_flag = True
    counter = 0
    while exist_flag and (samples_number==-1 or counter<samples_number):
        ret, frame = cap.read()
        if ret:
            test_frames.append(cv2.resize(frame,(300,300),interpolation = cv2.INTER_AREA))
            counter+=1
        
        if ret:
            exist_flag = True
        else:
            exist_flag = False
        
    cap.release()

    print("Load total number of video frames: ", len(test_frames))
    return test_frames


In [15]:
def convert_rgb_frame_to_tensor(image):
    img_size = 416
    # Configure input
    input_img = transforms.Compose([
    DEFAULT_TRANSFORMS,
    Resize(img_size)])(
        (image, np.zeros((1, 5))))[0].unsqueeze(0)
    input_img = input_img.cuda()

    return input_img

In [16]:
class cots_framework():

    def __init__(self, model,device) -> None:
        self.model = model
        self.device = device
        self.reference_tensor=None
        self.tensor_size= None
        self.tensor_shape = None
        self.pruning_threshold= None

        # Measurements
        self.diff_tensor_sparsity = []
        self.pruned_tensor_sparsity = []
        self.pruned_tensor_rank =[]
        self.data_size = []
        self.reconstruct_error = []
        self.head_time =[]
        self.framework_time = []
        self.tail_time = []
        self.start_event = torch.cuda.Event(enable_timing=True)
        self.end_event = torch.cuda.Event(enable_timing=True)

    def set_reference_tensor(self, test_tensor):
        head_tensor = self.model(test_tensor, 1)
        self.tensor_shape = head_tensor.shape
        self.reference_tensor = torch.zeros( self.tensor_shape, dtype=torch.float32).to(self.device)
        self.reference_tensor_edge = torch.zeros( self.tensor_shape, dtype=torch.float32).to(self.device)
        self.tensor_size = self.tensor_shape[0]*self.tensor_shape[1]*self.tensor_shape[2]*self.tensor_shape[3]

    def set_pruning_threshold(self, threshold):
        self.pruning_threshold = threshold
    
    def set_jpeg_quality(self, quality):
        self.jpeg_quality =  quality

    def calculate_snr(self,original_signal, reconstructed_signal):
        # Calculate the noise signal
        noise_signal = original_signal - reconstructed_signal
        
        # Calculate the power of the original signal
        P_signal = np.mean(np.square(original_signal))
        
        # Calculate the power of the noise signal
        P_noise = np.mean(np.square(noise_signal))
        
        # Calculate SNR in dB
        SNR_dB = 10 * np.log10(P_signal / P_noise)
        
        return SNR_dB
    
    def jpeg_encoding_decoding(self,tensor):
        # Normalize & Quantize Config
        normalize_base =torch.abs(tensor).max()
        tensor_normal = tensor / normalize_base
        scale, zero_point = (tensor_normal.max()-tensor_normal.min())/255,127
        dtype = torch.quint8

        
        tensor_normal = torch.quantize_per_tensor(tensor_normal, scale, zero_point, dtype)
        tensor_normal = tensor_normal.int_repr()
        tensor_normal = tensor_normal.to(torch.uint8).reshape((1,self.tensor_shape[1],self.tensor_shape[2]*self.tensor_shape[3]))

        # JPEG encoding/decoding
        transfer_data = tvio.encode_jpeg(tensor_normal.cpu(),self.jpeg_quality)
        self.data_size.append(transfer_data.shape[0])
        print(transfer_data.shape)
        received_data = tvio.decode_jpeg(transfer_data,device="cuda")

        # Reconstruct diff tensor
        reconstructed_tensor = received_data.reshape(self.tensor_shape)
        reconstructed_tensor = (reconstructed_tensor.to(torch.float)-zero_point) * scale * normalize_base
        snr = self.calculate_snr(reconstructed_tensor.reshape(self.tensor_size).cpu().numpy() , tensor.reshape(self.tensor_size).cpu().numpy())
        self.reconstruct_error.append(snr)
        return reconstructed_tensor


    
    def object_detection(self, tensor):
        with torch.no_grad():
            # Execute head model
            self.start_event.record()
            head_tensor = self.model(tensor, 1)
            self.end_event.record()
            torch.cuda.synchronize()
            self.head_time.append(self.start_event.elapsed_time(self.end_event))

            self.start_event.record()
            # diff operator
            diff_tensor = head_tensor-self.reference_tensor
            self.diff_tensor_sparsity.append(torch.sum(diff_tensor==0).cpu().item()/self.tensor_size)

            # pruner
            diff_tensor_normal = torch.nn.functional.normalize(diff_tensor)
            pruned_tensor = diff_tensor*(abs(diff_tensor_normal) > self.pruning_threshold)
            # self.reference_tensor = self.reference_tensor + pruned_tensor
            # print(pruned_tensor.shape)
            self.pruned_tensor_sparsity.append(torch.sum(pruned_tensor==0).cpu().item()/self.tensor_size)
            self.pruned_tensor_rank.append(torch.linalg.matrix_rank(pruned_tensor).cpu().numpy().mean())

            # Compressive sensing
            # compresssed_data= self.compressive_sensing_sample(pruned_tensor)
            # reconstructed_data = self.compressive_sensing_reconstruct(compresssed_data)
            reconstructed_data  = self.jpeg_encoding_decoding(pruned_tensor)

            # recosntruct intra model tensor and inference
            # self.reference_tensor = self.reference_tensor + reconstructed_data
            self.reference_tensor = self.reference_tensor + reconstructed_data
            self.end_event.record()
            torch.cuda.synchronize()
            self.framework_time.append(self.start_event.elapsed_time(self.end_event))

            self.start_event.record()
            inference_result = self.model(self.reference_tensor,2)
            self.end_event.record()
            torch.cuda.synchronize()
            self.tail_time.append(self.start_event.elapsed_time(self.end_event))
            detection = non_max_suppression(inference_result, 0.5, 0.5)

        return detection

### Model test

In [17]:
# Load the YOLO model
model = models_split_tiny.load_model(
  "../config/yolov3-tiny.cfg",
  "../weights/yolov3-tiny.weights")

model.set_split_layer(7) # layer <7
model = model.eval()


In [18]:
video_names= ["b4c0a877-66fc6239","b4d2dede-18dc0cd0","b4fe0b47-a7819060","b5f9cb1e-c84e2ce9","b5fa32f1-4e9c7d32",
              "b49a1b50-186fb520","b69d5b1d-03bd6daa","b607dd6a-1d8f33b9","b610204c-e3c8c65f"]

# video_names= ["b4c0a877-66fc6239","b4d2dede-18dc0cd0"]
video_dir = "/home/fullsuper/Rex/dataset/bdd100k/bdd100k/videos/val/"
map_output_path = "../bdd100k_measurements/deduplication/jpeg_map.csv"
perf_output_path = "../bdd100k_measurements/deduplication/jpeg_time.csv"
resource_output_path = "../bdd100k_measurements/deduplication/jpeg_resource.csv"
test_frame_len = 100

In [19]:
## Remove old files
# try:
#         os.remove(map_output_path)
#         os.remove(perf_output_path)
#         os.remove(resource_output_path)
# except:
#         print("File not exist")
        
# time.sleep(3)

# with open(map_output_path,'a') as f:
#         f.write("video_name,pruning_thresh,jepg_quality,map,map_50,map_75,map_small,map_medium, ap_large,mar_1,mar_100,mar_small,mar_medium,mar_large\n")

# with open(perf_output_path,'a') as f:
#         f.write("video_name,pruning_thresh,jepg_quality,snr_mean,snr_std,head_time_mean,head_time_std,framework_time_mean,framework_time_std,tail_time_mean,tail_time_std,data_size_mean,data_size_std\n")
# with open(resource_output_path,'a') as f:
#         f.write("video_name,pruning_thresh,jepg_quality,cpu_total_mean,cpu_total_std,cuda_total_mean,cuda_total_std,cpu_mem_mean,cpu_mem_std,cuda_mem_mean,cuda_mem_std\n")

In [20]:

for video_name in video_names:
    print("Testing with video: "+video_name)
    test_frames = load_video_frames(video_dir,video_name, test_frame_len)
    frame_labels = load_ground_truth(video_name)

    ## Test different configurations
    for i in range(5):
        print("In iter",i)
        frame_predicts = []
        thresh = 0.05
        quality = 60+10*i


        cots = cots_framework(model,device="cuda")
        cots.set_reference_tensor(convert_rgb_frame_to_tensor(test_frames[0]))
        cots.set_pruning_threshold(thresh)
        cots.set_jpeg_quality(quality)

        indexes = []
        cpu_time = []
        cuda_time =[]
        cpu_mem = []
        cuda_mem = []
        for index in range(len(test_frames)):
        # for index in range(70,70+samples_number):
        # for index in range(3):
            frame = test_frames[index]
            frame = convert_rgb_frame_to_tensor(frame)
            with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True) as prof:
                with record_function("model_inference"):
                    detection = cots.object_detection(frame)
            resource_mea = prof.key_averages().table(sort_by="cuda_time_total", row_limit=1)
            mea=list(filter(None,resource_mea.split('\n')[3].split(" ")) ) 
            cpu_time.append(float(str(mea[4]).replace("m","").replace("s","")))
            cuda_time.append(float(str(mea[8]).replace("m","").replace("s","")))
            cpu_mem.append(abs(float(mea[12]))/1000 if mea[13]=='Kb' else abs(float(mea[12])))
            cuda_mem.append(abs(float(mea[16]))/1000 if mea[17] =='Kb' else abs(float(mea[16])))
            # print(detection)
            return
            # [[x1, y1, x2, y2, confidence, class]]
            if len(detection[0])!= 0:
                pred = dict(boxes=tensor(detection[0].numpy()[:,0:4]),
                            scores=tensor(detection[0].numpy()[:,4]),
                            labels=tensor(detection[0].numpy()[:,5],dtype=torch.int32), )
            else:
                pred = dict(boxes=tensor([[-1.0,-1.0,-1.0,-1.0]]),
                            scores=tensor([-1.0]),
                            labels=tensor([-1],dtype=torch.int32), )

            frame_predicts.append(pred)
        
        metric = MeanAveragePrecision(iou_type="bbox")  
        metric.update(frame_predicts, frame_labels[0:test_frame_len])
        maps = metric.compute()

        with open(map_output_path,'a') as f:
            f.write(video_name+","
                    +str(thresh)+","
                    +str(quality)+","
                    +str(maps["map"].item())+","
                    +str(maps["map_50"].item())+","
                    +str(maps["map_75"].item())+","
                    +str(maps["map_small"].item())+","
                    +str(maps["map_medium"].item())+","
                    +str(maps["map_large"].item())+","
                    +str(maps["mar_1"].item())+","
                    +str(maps["mar_100"].item())+","
                    +str(maps["mar_small"].item())+","
                    +str(maps["mar_medium"].item())+","
                    +str(maps["mar_large"].item())+"\n"
                    )
        
        with open(perf_output_path,'a') as f:
            f.write(video_name+","
                    +str(thresh)+","
                    +str(quality)+","
                    +str(np.array(cots.reconstruct_error).mean())+","
                    +str(np.array(cots.reconstruct_error).std())+","
                    +str(np.array(cots.head_time).mean())+","
                    +str(np.array(cots.head_time).std())+","
                    +str(np.array(cots.framework_time).mean())+","
                    +str(np.array(cots.framework_time).std())+","
                    +str(np.array(cots.tail_time).mean())+","
                    +str(np.array(cots.tail_time).std())+","
                    +str(np.array(cots.data_size).mean())+","
                    +str(np.array(cots.data_size).std())+"\n"
                    )
            
        with open(resource_output_path,'a') as f:
            f.write(video_name+","
                    +str(thresh)+","
                    +str(quality)+","
                    +str(np.array(cpu_time).mean())+","
                    +str(np.array(cpu_time).std())+","
                    +str(np.array(cuda_time).mean())+","
                    +str(np.array(cuda_time).std())+","
                    +str(np.array(cpu_mem).mean())+","
                    +str(np.array(cpu_mem).std())+","
                    +str(np.array(cuda_mem).mean())+","
                    +str(np.array(cuda_mem).std())+"\n"
            )

Testing with video: b4c0a877-66fc6239
Load total number of video frames:  100


/tmp/ipykernel_1397675/1895112543.py:14: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  frame_labels.append(dict(boxes=tensor(boxes,dtype=torch.float32),labels=tensor(labels,dtype=torch.int32),) )
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing


In iter 0


STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 16:53:05 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2024-04-18 16:53:05 1397675:139767

In iter 1


...

STAGE:2024-04-18 17:02:08 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 17:02:08 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 17:02:08 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2024-04-18 17:02:08 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 17:02:08 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 17:02:08 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2024-04-18 17:02:09 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 17:02:09 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 17:02:09 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2024-04-18 17:02:09 1397675:139767

In iter 3


STAGE:2024-04-18 17:02:22 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 17:02:22 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 17:02:22 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2024-04-18 17:02:22 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 17:02:22 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 17:02:22 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2024-04-18 17:02:22 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 17:02:22 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 17:02:22 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2024-04-18 17:02:22 1397675:139767

In iter 4


STAGE:2024-04-18 17:02:35 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 17:02:35 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 17:02:35 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2024-04-18 17:02:35 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 17:02:35 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 17:02:35 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2024-04-18 17:02:36 1397675:1397675 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-04-18 17:02:36 1397675:1397675 ActivityProfilerController.cpp:318] Completed Stage: Collection
STAGE:2024-04-18 17:02:36 1397675:1397675 ActivityProfilerController.cpp:322] Completed Stage: Post Processing
STAGE:2024-04-18 17:02:36 1397675:139767